# Getting Started with LangGraph Tools

## Objective
Learn the fundamentals of creating tools in LangGraph that extend LLM capabilities.

## What You'll Learn
1. What tools are and why they matter
2. How to create tools using the `@tool` decorator
3. The difference between regular functions and LangGraph tools
4. How to invoke tools correctly
5. Understanding tool schemas (what LLMs see)

## Prerequisites
- Basic Python knowledge
- Understanding of functions and decorators

---

## Section 1: What Are Tools?

**Tools** are functions that extend what an LLM can do. While LLMs are great at understanding and generating text, they can't:
- Perform precise calculations
- Access real-time data (APIs, databases)
- Interact with external systems

Tools bridge this gap by providing LLMs with callable functions that perform specific tasks.

### How Tools Work in LangGraph
1. You define a tool with clear documentation
2. The LLM receives the tool's **schema** (name, description, parameters)
3. When needed, the LLM requests a tool call with specific arguments
4. LangGraph's `ToolNode` executes the actual function
5. The result is returned to the LLM for interpretation

> **Key Insight:** The LLM never runs the tool directly—it only *requests* the tool call. LangGraph handles the actual execution.

---

## Section 2: Setup

We'll import the `@tool` decorator from `langchain_core.tools`. This is the foundation for creating LangGraph-compatible tools.

In [ ]:
# Core import for creating tools
from langchain_core.tools import tool

# For pretty-printing schemas
from pprint import pprint

print("✅ Imports successful")

---

## Section 3: Creating Your First Tool

Let's create a simple greeting tool. Notice the three essential components:

1. **`@tool` decorator** - Transforms the function into a LangGraph tool
2. **Type hints** - Define parameter and return types (required!)
3. **Docstring** - Describes what the tool does (LLM reads this!)

In [ ]:
@tool
def greet(name: str) -> str:
    """Greet a person by name.
    
    Use this tool when you need to generate a personalized greeting message.
    """
    return f"Hello, {name}! Welcome to our assistant."

print("✅ greet tool defined")
print(f"   Tool Name: {greet.name}")
print(f"   Tool Type: {type(greet)}")

### Reference Point: Tool Properties

After decoration, your function becomes a `StructuredTool` object with properties:
- `.name` - The tool's identifier
- `.description` - Extracted from the docstring
- `.args_schema` - Pydantic model defining the input schema

---

## Section 4: Invoking Tools - The Right Way

### ❌ This Will Fail

A common mistake is calling the tool like a regular function:

In [ ]:
# ❌ INCORRECT: Calling tool like a regular function
# This will raise a ValidationError!

try:
    result = greet("Alice")
    print(result)
except Exception as e:
    print(f"❌ Error Type: {type(e).__name__}")
    print(f"   Message: {e}")

### ✅ The Correct Way: Using `.invoke()`

LangGraph tools must be invoked with a **dictionary** of arguments using the `.invoke()` method:

In [ ]:
# ✅ CORRECT: Using .invoke() with a dictionary
result = greet.invoke({"name": "Alice"})
print(result)

### Reference Point: Why `.invoke()` with a Dictionary?

This pattern exists because:
1. **LLMs output JSON** - When an LLM calls a tool, it generates arguments as a JSON object
2. **Consistency** - The same invocation pattern works across all LangChain/LangGraph components
3. **Validation** - The dictionary is validated against the tool's schema before execution

---

## Section 5: Understanding Tool Schemas

The **schema** is what the LLM sees when deciding whether to use a tool. It's automatically generated from your function's:
- Type hints
- Docstring
- Parameter names

Let's examine the schema for our `greet` tool:

In [ ]:
# View the JSON schema that LLMs use to understand this tool
print("Schema for 'greet' tool:")
print("=" * 50)
greet_schema = greet.args_schema.model_json_schema()
pprint(greet_schema)

### Reference Point: Schema Components

| Field | Description |
|-------|-------------|
| `title` | Derived from function name |
| `description` | From docstring (first line) |
| `properties` | Dictionary of parameters |
| `required` | List of required parameters |
| `type` | Always "object" for tool inputs |

---

## Section 6: Tool with Multiple Parameters

Let's create a tool with multiple parameters to see how the schema changes:

In [ ]:
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two integers together.
    
    Use this tool for simple addition operations.
    
    Args:
        a: The first number to add
        b: The second number to add
    
    Returns:
        The sum of a and b
    """
    return a + b

print("✅ add_numbers tool defined")
print(f"   Tool Name: {add_numbers.name}")

In [ ]:
# Test the tool
result = add_numbers.invoke({"a": 5, "b": 3})
print(f"5 + 3 = {result}")

In [ ]:
# View the schema - notice both parameters are required
print("Schema for 'add_numbers' tool:")
print("=" * 50)
pprint(add_numbers.args_schema.model_json_schema())

### Reference Point: Type Hints Matter!

Notice how `int` type hints become `"type": "integer"` in the schema. Common mappings:

| Python Type | JSON Schema Type |
|-------------|------------------|
| `str` | `string` |
| `int` | `integer` |
| `float` | `number` |
| `bool` | `boolean` |
| `list` | `array` |
| `dict` | `object` |

---

## Section 7: Best Practices for Tool Creation

When creating tools, follow these guidelines:

### 1. Write Clear Docstrings
The LLM uses your docstring to decide when to use the tool. Be specific!

```python
# ❌ Vague
"""Do something with numbers"""

# ✅ Clear
"""Add two integers together. Use for simple addition operations."""
```

### 2. Use Descriptive Parameter Names
```python
# ❌ Unclear
def calc(x: float, y: float, z: int)

# ✅ Clear
def calculate_emi(principal: float, interest_rate: float, tenure_months: int)
```

### 3. Always Include Type Hints
Without type hints, the schema won't include parameter types, making it harder for the LLM to provide correct values.

### 4. Return Human-Readable Strings
When the result goes back to the LLM, strings are easiest to interpret and relay to users.

---

## Summary

In this notebook, you learned:

| Concept | Key Takeaway |
|---------|-------------|
| **Tools** | Functions that extend LLM capabilities |
| **@tool decorator** | Transforms functions into LangGraph tools |
| **Invocation** | Use `.invoke({"param": value})`, not direct calls |
| **Schemas** | Auto-generated from type hints and docstrings |
| **Best Practices** | Clear docstrings, descriptive names, type hints |

## Next Steps

In the following notebooks, we'll build practical tools:
1. **Currency Converter** - Multi-parameter tool with validation
2. **EMI Calculator** - Complex calculations with financial formulas

---

## Practice Exercises

Try creating these tools on your own:

1. **Multiply Tool** - Takes two numbers and returns their product
2. **Greeting Tool** - Takes a name and language, returns greeting in that language
3. **Temperature Converter** - Converts between Celsius and Fahrenheit

In [ ]:
# Exercise 1: Create a multiply tool
# Your code here:



In [ ]:
# Exercise 2: Create a multi-language greeting tool
# Your code here:



In [ ]:
# Exercise 3: Create a temperature converter tool
# Your code here:

